##### ANALYZING PITCH DRIVE(angle of the rotor blades) AND ICING EVENTS 

Models (Focus on  Anomaly detection & Early warnings)
1. CNN + LSTM   
2. LSTM Autoencoder  
 

##### SENSOR METADATA STRUCTURE

sampling_rate = 200


In [ ]:
import h5py



def h5_tree_to_dict(val):
    """
    This function will recursively traverse the h5py object and return a dictionary-like object representing the tree structure. The keys of the dictionary will be the names of the groups and datasets in the h5py object, and the values will either be integers (representing the length of a dataset) or nested dictionary-like objects (representing a group)."
    :param val: HDF5 file
    :return dict: Dictionary with structure metadata
    """
    result = {}
    items = len(val)
    for key, val in val.items():
        items -= 1
        if items == 0:
            # the last item
            if type(val) == h5py._hl.group.Group:
                result[key] = h5_tree_to_dict(val)
            else:
                result[key] = len(val)
        else:
            if type(val) == h5py._hl.group.Group:
                result[key] = h5_tree_to_dict(val)
            else:
                result[key] = len(val)
    return result

def h5_tree(val, pre=''):
    """
    This function will recursively traverse the h5py object and print out the tree structure
    """
    items = len(val)
    for key, val in val.items():
        items -= 1
        if items == 0:
            # the last item
            if type(val) == h5py._hl.group.Group:
                print(pre + '└── ' + key)
                h5_tree(val, pre+'    ')
            else:
                print(pre + '└── ' + key + ' (%d)' % len(val))
        else:
            if type(val) == h5py._hl.group.Group:
                print(pre + '├── ' + key)
                h5_tree(val, pre+'│   ')
            else:
                print(pre + '├── ' + key + ' (%d)' % len(val))

##### STRUCTURE OF THE DATA

In [ ]:
import h5py    

SAMPLE_FILE = r"C:\Users\KIIT\AppData\Local\Temp\a6c706f0-4ff4-4444-94df-229141d86dae_aventa_rotor_icing.zip.dae\aventa_rotor_icing\Aventa_Taggenberg_01_11_2022.hdf5"
with h5py.File(SAMPLE_FILE, 'r') as hf:
    print(hf)
    h5_tree(hf)

##### CONVERT TO JSON

In [ ]:
import json
import h5py
FILE_PATH = r"C:\Users\KIIT\AppData\Local\Temp\a6c706f0-4ff4-4444-94df-229141d86dae_aventa_rotor_icing.zip.dae\aventa_rotor_icing\Aventa_Taggenberg_01_11_2022.hdf5"


file_name = FILE_PATH.split(".")[0].split("\\")[-1]

with h5py.File(SAMPLE_FILE, 'r') as hf:
    h5_tree_dict=h5_tree_to_dict(hf)
with open("{}_structure_metadata.json.".format(file_name), "w") as dump_file:
  json.dump(h5_tree_dict, dump_file)
  
  

##### EXTRACT THE SIGNAL (time and value recorded) INTO PANDAS DATA FRAME

In [41]:
import h5py
import pandas as pd
import plotly.express as px



FILE_PATH = r"C:\Users\KIIT\AppData\Local\Temp\a6c706f0-4ff4-4444-94df-229141d86dae_aventa_rotor_icing.zip.dae\aventa_rotor_icing\Aventa_Taggenberg_01_11_2022.hdf5"

import h5py

datasets = {}

with h5py.File(FILE_PATH, "r") as f:
    for dataset_name in f.keys():
      datasets[dataset_name]={}
      for time_stamp in f[dataset_name].keys():
        datasets[dataset_name][time_stamp]={}
        signal_list=list(f[dataset_name][time_stamp].keys())
        signal_list.remove('ChannelList')
        for signal in signal_list:
          datasets[dataset_name][time_stamp][signal]={}
          datasets[dataset_name][time_stamp][signal]['Time']=f[dataset_name][time_stamp][signal]['Time'][()]
          datasets[dataset_name][time_stamp][signal]['Value']=f[dataset_name][time_stamp][signal]['Value'][()]


TIME_STAMP='11_06_44'
CHANNEL = 'GEN_ACC_XX_01'

df=pd.DataFrame(
    {
        "Time":datasets['Aventa'][TIME_STAMP][CHANNEL]['Time'].flatten().tolist(),
        "Value":datasets['Aventa'][TIME_STAMP][CHANNEL]['Value'].flatten().tolist()
    }
)

fig=px.line(df, x="Time", y="Value")
fig.show()


